In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import json_normalize
import ast

In [2]:
movies_df = pd.read_csv("ETL_movies_main.csv")

In [3]:
# Eliminando duplicados porque afectan a revenue y valores de conteos o sumas
movies_df = movies_df.drop_duplicates(keep="first")

In [4]:
movies_df.shape

(45346, 20)

In [6]:
# getting columns needed
df_productions = movies_df.loc[:,["Movie_id","production_companies","production_countries"]]

In [7]:
# Turning NaN values into empty lists
df_productions['production_companies'] = df_productions['production_companies'].fillna('[]')

In [8]:
# str to dict
df_productions['production_companies'] = df_productions['production_companies'].apply(ast.literal_eval)

In [9]:
# exploding and checking
df_exploded = df_productions.explode('production_companies', ignore_index=True)
df_exploded

,Movie_id,production_companies,production_countries
0,862,"{'name': 'Pixar Animation Studios', 'id': 3}","[{'iso_3166_1': 'US', 'name': 'United States o..."
1,8844,"{'name': 'TriStar Pictures', 'id': 559}","[{'iso_3166_1': 'US', 'name': 'United States o..."
2,8844,"{'name': 'Teitler Film', 'id': 2550}","[{'iso_3166_1': 'US', 'name': 'United States o..."
3,8844,"{'name': 'Interscope Communications', 'id': 10...","[{'iso_3166_1': 'US', 'name': 'United States o..."
4,15602,"{'name': 'Warner Bros.', 'id': 6194}","[{'iso_3166_1': 'US', 'name': 'United States o..."
...,...,...,...
82233,30840,"{'name': 'CanWest Global Communications', 'id'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."
82234,111109,"{'name': 'Sine Olivia', 'id': 19653}","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]"
82235,67758,"{'name': 'American World Pictures', 'id': 6165}","[{'iso_3166_1': 'US', 'name': 'United States o..."
82236,227506,"{'name': 'Yermoliev', 'id': 88753}","[{'iso_3166_1': 'RU', 'name': 'Russia'}]"


In [10]:
# Separating names and ids
df_exploded['company_name'] = df_exploded['production_companies'].apply(lambda x: x.get('name', '') if isinstance(x, dict) else '')
df_exploded['company_id'] = df_exploded['production_companies'].apply(lambda x: x.get('id', '') if isinstance(x, dict) else '')

In [11]:
# Joining exploded with original df
df_productions = df_productions.join(df_exploded[['company_name', 'company_id']])

In [12]:
df_productions

,Movie_id,production_companies,production_countries,company_name,company_id
0,862,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",Pixar Animation Studios,3
1,8844,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",TriStar Pictures,559
2,15602,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",Teitler Film,2550
3,31357,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",Interscope Communications,10201
4,11862,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",Warner Bros.,6194
...,...,...,...,...,...
45341,30840,"[{'name': 'Westdeutscher Rundfunk (WDR)', 'id'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",Trimark Pictures,4063
45342,111109,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",Haut et Court,726
45343,67758,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",Mille et Une Productions,2566
45344,227506,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",Propaganda Pictures,2682


In [13]:
# dropping unnecesary column
df_productions.drop(["production_companies"],axis=1,inplace=True)

In [14]:
# Transforming NaN values into empty lists
df_productions['production_countries'] = df_productions['production_countries'].fillna('[]')

In [15]:
# str to list of dictionaries
df_productions['production_countries'] = df_productions['production_countries'].apply(ast.literal_eval)

In [16]:
# Unnesting/exploding production_countries
df_exploded2 = df_productions.explode('production_countries')

def extract_country_info(x):
    if isinstance(x, dict):
        return pd.Series([x.get('iso_3166_1', ''), x.get('name', '')])
    else:
        return pd.Series(['', ''])

df_exploded2[['country_code', 'country_name']] = df_exploded2['production_countries'].apply(extract_country_info)

In [17]:
# Joining df_exploded to original df
df_productions = df_productions.join(df_exploded2[['country_code', 'country_name']])

In [18]:
# Dropping unnecesary columns
df_productions.drop(["production_countries"],axis=1,inplace=True)

In [19]:
df_productions

,Movie_id,company_name,company_id,country_code,country_name
0,862,Pixar Animation Studios,3,US,United States of America
1,8844,TriStar Pictures,559,US,United States of America
2,15602,Teitler Film,2550,US,United States of America
3,31357,Interscope Communications,10201,US,United States of America
4,11862,Warner Bros.,6194,US,United States of America
...,...,...,...,...,...
45341,30840,Trimark Pictures,4063,US,United States of America
45342,111109,Haut et Court,726,PH,Philippines
45343,67758,Mille et Une Productions,2566,US,United States of America
45344,227506,Propaganda Pictures,2682,RU,Russia


In [24]:
# It seems like the Movies are been made in more than 1 country
df_productions.sort_values('Movie_id',ascending=True)

,Movie_id,company_name,company_id,country_code,country_name
4337,2,"Ministry of Film, The",21085,FI,Finland
12938,3,August Entertainment,3322,FI,Finland
17,5,Worldwide,55873,US,United States of America
474,6,Universal Pictures,33,US,United States of America
474,6,Universal Pictures,33,JP,Japan
...,...,...,...,...,...
45156,467731,Red Wagon Entertainment,14440,US,United States of America
21854,468343,France2 Cinéma,7089,FI,Finland
45280,468707,Finos Films,12991,FI,Finland
20161,469172,Rhône-Alpes Cinéma,1666,FR,France


In [24]:
# df to csv

In [25]:
df_productions.to_csv("ETL_movies_productions.csv",index=False)